# Utils

In [ ]:
import glob
import os
import random
from datetime import datetime
from pathlib import Path

import laspy
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch
from sklearn.metrics import confusion_matrix
from torch_geometric.data import Data, InMemoryDataset

In [ ]:
def read_las(pointcloudfile, get_attributes=False, useevery=1):
    """
    :param pointcloudfile: specification of input file (format: las or laz)
    :param get_attributes: if True, will return all attributes in file, otherwise will only return XYZ (default is False)
    :param useevery: value specifies every n-th point to use from input, i.e. simple subsampling (default is 1, i.e. returning every point)
    :return: 3D array of points (x,y,z) of length number of points in input file (or subsampled by 'useevery')
    """

    # Read file
    inFile = laspy.read(pointcloudfile)

    # Get coordinates (XYZ)
    coords = np.vstack((inFile.x, inFile.y, inFile.z)).transpose()
    coords = coords[::useevery, :]

    # Return coordinates only
    if get_attributes == False:
        return coords

    # Return coordinates and attributes
    else:
        las_fields = [info.name for info in inFile.points.point_format.dimensions]
        attributes = {}
        # for las_field in las_fields[3:]:  # skip the X,Y,Z fields
        for las_field in las_fields:  # get all fields
            attributes[las_field] = inFile.points[las_field][::useevery]
        return (coords, attributes)

In [ ]:
def rotate_points(coords):
    rotation = np.random.uniform(-180, 180)
    # Convert rotation values to radians
    rotation = np.radians(rotation)

    # Rotate point cloud
    rot_mat = np.array(
        [
            [np.cos(rotation), -np.sin(rotation), 0],
            [np.sin(rotation), np.cos(rotation), 0],
            [0, 0, 1],
        ]
    )

    aug_coords = coords
    aug_coords[:, :3] = np.matmul(aug_coords[:, :3], rot_mat)
    return aug_coords

In [ ]:
def point_removal(coords, x=None):
    # Get list of ids
    idx = list(range(np.shape(coords)[0]))
    random.shuffle(idx)  # shuffle ids
    idx = np.random.choice(
        idx, random.randint(len(idx) - 50, len(idx)), replace=False
    )  # pick points randomly removing 0 - 50 points

    # Remove random values
    aug_coords = coords[idx, :]  # remove coords
    if x is None:  # remove x
        aug_x = aug_coords
    else:
        aug_x = x[idx, :]

    return aug_coords, aug_x

In [ ]:
def random_noise(coords, dim, x=None):
    # Random standard deviation value
    random_noise_sd = np.random.uniform(0.01, 0.025)

    # Add/Subtract noise
    if np.random.uniform(0, 1) >= 0.5:  # 50% chance to add
        aug_coords = coords + np.random.normal(
            0, random_noise_sd, size=(np.shape(coords)[0], 3)
        )
        if x is None:
            aug_x = aug_coords
        else:
            aug_x = x + np.random.normal(0, random_noise_sd, size=(np.shape(x)))
    else:  # 50% chance to subtract
        aug_coords = coords - np.random.normal(
            0, random_noise_sd, size=(np.shape(coords)[0], 3)
        )
        if x is None:
            aug_x = aug_coords
        else:
            aug_x = x - np.random.normal(0, random_noise_sd, size=(np.shape(x)))

    # Randomly choose between 0 and 50 augmented noise points
    use_idx = np.random.choice(
        aug_coords.shape[0], random.randint(0, 50), replace=False
    )
    aug_coords = aug_coords[use_idx, :]  # get random points
    aug_coords = np.append(coords, aug_coords, axis=0)  # add points
    aug_x = aug_x[use_idx, :]  # get random point values
    aug_x = np.append(x, aug_x)  # add random point values

    if dim == 1:
        aug_x = aug_x[:, np.newaxis]

    return aug_coords, aug_x

In [ ]:
class PointCloudsInFiles(InMemoryDataset):
    """Point cloud dataset where one data point is a file."""

    def __init__(
        self, root_dir, glob="*", column_name="", max_points=200_000, use_columns=None
    ):
        """
        Args:
            root_dir (string): Directory with the datasets
            glob (string): Glob string passed to pathlib.Path.glob
            column_name (string): Column name to use as target variable (e.g. "Classification")
            use_columns (list[string]): Column names to add as additional input
        """
        self.files = list(Path(root_dir).glob(glob))
        self.column_name = column_name
        self.max_points = max_points
        if use_columns is None:
            use_columns = []
        self.use_columns = use_columns
        super().__init__()

    def __len__(self):
        # Return length
        return len(self.files)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        # Get file name
        filename = str(self.files[idx])

        # Read las/laz file
        coords, attrs = read_las(filename, get_attributes=True)

        # Resample number of points to max_points
        if coords.shape[0] >= self.max_points:
            use_idx = np.random.choice(coords.shape[0], self.max_points, replace=False)
        else:
            use_idx = np.random.choice(coords.shape[0], self.max_points, replace=True)

        # Get x values
        if len(self.use_columns) > 0:
            x = np.empty((self.max_points, len(self.use_columns)), np.float32)
            for eix, entry in enumerate(self.use_columns):
                x[:, eix] = attrs[entry][use_idx]
        else:
            x = coords[use_idx, :]

        # Get coords
        coords = coords - np.mean(coords, axis=0)  # centralize coordinates

        # impute target
        target = attrs[self.column_name]
        target[np.isnan(target)] = np.nanmean(target)

        # Transform data to tensor
        sample = Data(
            x=torch.from_numpy(x).float(),
            y=torch.from_numpy(
                np.unique(np.array(target[use_idx][:, np.newaxis]))
            ).type(torch.LongTensor),
            pos=torch.from_numpy(coords[use_idx, :]).float(),
        )
        if coords.shape[0] < 100:
            return None
        return sample

In [ ]:
class AugmentPointCloudsInFiles(InMemoryDataset):
    """Point cloud dataset where one data point is a file."""

    def __init__(
        self, root_dir, glob="*", column_name="", max_points=200_000, use_columns=None
    ):
        """
        Args:
            root_dir (string): Directory with the datasets
            glob (string): Glob string passed to pathlib.Path.glob
            column_name (string): Column name to use as target variable (e.g. "Classification")
            use_columns (list[string]): Column names to add as additional input
        """
        self.files = list(Path(root_dir).glob(glob))
        self.column_name = column_name
        self.max_points = max_points
        if use_columns is None:
            use_columns = []
        self.use_columns = use_columns
        super().__init__()

    def __len__(self):
        # Return length
        return len(self.files)  # NEED TO ADD MULTIPLICATION FOR NUMBER OF AUGMENTS

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        # Get file name
        filename = str(self.files[idx])

        # Read las/laz file
        coords, attrs = read_las(filename, get_attributes=True)

        # Resample number of points to max_points
        if coords.shape[0] >= self.max_points:
            use_idx = np.random.choice(coords.shape[0], self.max_points, replace=False)
        else:
            use_idx = np.random.choice(coords.shape[0], self.max_points, replace=True)

        # Get x values
        if len(self.use_columns) > 0:
            x = np.empty((self.max_points, len(self.use_columns)), np.float32)
            for eix, entry in enumerate(self.use_columns):
                x[:, eix] = attrs[entry][use_idx]
        else:
            x = coords[use_idx, :]

        # Get coords
        coords = coords[use_idx, :]
        coords = coords - np.mean(coords, axis=0)  # centralize coordinates

        # Augmentation
        coords, x = point_removal(coords, x)
        coords, x = random_noise(coords, len(self.use_columns), x)
        coords = rotate_points(coords)

        # impute target
        target = attrs[self.column_name]
        target[np.isnan(target)] = np.nanmean(target)

        # Transform data to tensor
        sample = Data(
            x=torch.from_numpy(x).float(),
            y=torch.from_numpy(np.unique(np.array(target[:, np.newaxis]))).type(
                torch.LongTensor
            ),
            pos=torch.from_numpy(coords).float(),
        )
        if coords.shape[0] < 100:
            return None
        return sample

In [ ]:
class IOStream:
    def __init__(self, path):
        # Open file in append
        self.f = open(path, "a")

    def cprint(self, text):
        # Print and write text to file
        print(text)  # print text
        self.f.write(text + "\n")  # write text and new line
        self.f.flush  # flush file

    def close(self):
        sefl.f.close()  # close file

In [ ]:
def _init_(model_name):
    # Create folder structure
    if not os.path.exists("checkpoints"):
        os.makedirs("checkpoints")
    if not os.path.exists("checkpoints/" + model_name):
        os.makedirs("checkpoints/" + model_name)
    if not os.path.exists("checkpoints/" + model_name + "/models"):
        os.makedirs("checkpoints/" + model_name + "/models")
    if not os.path.exists("checkpoints/" + model_name + "/confusion_matrix"):
        os.makedirs("checkpoints/" + model_name + "/confusion_matrix")
    if not os.path.exists("checkpoints/" + model_name + "/confusion_matrix/all"):
        os.makedirs("checkpoints/" + model_name + "/confusion_matrix/all")
    if not os.path.exists("checkpoints/" + model_name + "/confusion_matrix/best"):
        os.makedirs("checkpoints/" + model_name + "/confusion_matrix/best")
    if not os.path.exists("checkpoints/" + model_name + "/classification_report"):
        os.makedirs("checkpoints/" + model_name + "/classification_report")
    if not os.path.exists("checkpoints/" + model_name + "/classification_report/all"):
        os.makedirs("checkpoints/" + model_name + "/classification_report/all")
    if not os.path.exists("checkpoints/" + model_name + "/classification_report/best"):
        os.makedirs("checkpoints/" + model_name + "/classification_report/best")

In [ ]:
def make_confusion_matrix(
    cf,
    group_names=None,
    categories="auto",
    count=True,
    percent=True,
    cbar=True,
    xyticks=True,
    xyplotlabels=True,
    sum_stats=True,
    figsize=None,
    cmap="Blues",
    title=None,
):
    """
    This function will make a pretty plot of an sklearn Confusion Matrix cm using a Seaborn heatmap visualization.
    Arguments
    ---------
    cf:            confusion matrix to be passed in

    group_names:   List of strings that represent the labels row by row to be shown in each square.

    categories:    List of strings containing the categories to be displayed on the x,y axis. Default is 'auto'

    count:         If True, show the raw number in the confusion matrix. Default is True.

    percent:     If True, show the proportions for each category. Default is True.

    cbar:          If True, show the color bar. The cbar values are based off the values in the confusion matrix.
                   Default is True.

    xyticks:       If True, show x and y ticks. Default is True.

    xyplotlabels:  If True, show 'True Label' and 'Predicted Label' on the figure. Default is True.

    sum_stats:     If True, display summary statistics below the figure. Default is True.

    figsize:       Tuple representing the figure size. Default will be the matplotlib rcParams value.

    cmap:          Colormap of the values displayed from matplotlib.pyplot.cm. Default is 'Blues'
                   See http://matplotlib.org/examples/color/colormaps_reference.html

    title:         Title for the heatmap. Default is None.
    """

    blanks = ["" for i in range(cf.size)]

    if group_names and len(group_names) == cf.size:
        group_labels = ["{}\n".format(value) for value in group_names]
    else:
        group_labels = blanks

    if count:
        group_counts = ["{0:0.0f}\n".format(value) for value in cf.flatten()]
    else:
        group_counts = blanks

    if percent:
        group_percentages = [
            "{0:.2%}".format(value) for value in cf.flatten() / np.sum(cf)
        ]
    else:
        group_percentages = blanks

    box_labels = [
        f"{v1}{v2}{v3}".strip()
        for v1, v2, v3 in zip(group_labels, group_counts, group_percentages)
    ]
    box_labels = np.asarray(box_labels).reshape(cf.shape[0], cf.shape[1])

    # CODE TO GENERATE SUMMARY STATISTICS & TEXT FOR SUMMARY STATS
    if sum_stats:
        # Accuracy is sum of diagonal divided by total observations
        accuracy = np.trace(cf) / float(np.sum(cf))

        # if it is a binary confusion matrix, show some more stats
        if len(cf) == 2:
            # Metrics for Binary Confusion Matrices
            precision = cf[1, 1] / sum(cf[:, 1])
            recall = cf[1, 1] / sum(cf[1, :])
            f1_score = 2 * precision * recall / (precision + recall)
            stats_text = "\n\nAccuracy={:0.3f}\nPrecision={:0.3f}\nRecall={:0.3f}\nF1 Score={:0.3f}".format(
                accuracy, precision, recall, f1_score
            )
        else:
            stats_text = "\n\nAccuracy={:0.3f}".format(accuracy)
    else:
        stats_text = ""

    # SET FIGURE PARAMETERS ACCORDING TO OTHER ARGUMENTS
    if figsize == None:
        # Get default figure size if not set
        figsize = plt.rcParams.get("figure.figsize")

    if xyticks == False:
        # Do not show categories if xyticks is False
        categories = False

    # MAKE THE HEATMAP VISUALIZATION
    plt.figure(figsize=figsize)
    sns.heatmap(
        cf,
        annot=box_labels,
        fmt="",
        cmap=cmap,
        cbar=cbar,
        xticklabels=categories,
        yticklabels=categories,
    )

    if xyplotlabels:
        plt.ylabel("True label")
        plt.xlabel("Predicted label" + stats_text)
    else:
        plt.xlabel(stats_text)

    if title:
        plt.title(title)

In [ ]:
def delete_files(root_dir, glob="*"):
    # List files in root_dir with glob
    files = list(Path(root_dir).glob(glob))

    # Delete files
    for f in files:
        os.remove(f)

# PointCNN

In [ ]:
import torch
import torch.nn.functional as F
from torch import nn
from torch.optim import Adam
from torch_geometric.nn import XConv, fps, global_mean_pool

In [ ]:
class PointCNN(nn.Module):
    def __init__(self, numfeatures, numclasses):
        super().__init__()
        self.numfeatures = numfeatures
        self.numclasses = numclasses

        # First XConv layer
        self.conv1 = XConv(
            self.numfeatures, 48, dim=3, kernel_size=8, hidden_channels=32
        )

        # Second XConv layer
        self.conv2 = XConv(
            48, 96, dim=3, kernel_size=12, hidden_channels=64, dilation=2
        )

        # Third XConv layer
        self.conv3 = XConv(
            96, 192, dim=3, kernel_size=16, hidden_channels=128, dilation=2
        )

        # Fourth XConv layer
        self.conv4 = XConv(
            192, 384, dim=3, kernel_size=16, hidden_channels=256, dilation=2
        )

        # Multilayer Perceptrons (MLPs) at the end of the PointCNN
        self.lin1 = nn.Linear(384, 256)
        self.lin2 = nn.Linear(256, 128)
        self.lin3 = nn.Linear(
            128, self.numclasses
        )  # change last value for number of classes

    def forward(self, data):
        # Get pos and batch
        pos, batch = data.pos, data.batch

        # Get x
        x = data.x if self.numfeatures else None

        # First XConv with no features
        x = F.relu(self.conv1(x, pos, batch))
        # x = torch.nn.ReLU(self.conv1(x, pos, batch))

        # Farthest point sampling, keeping only 37.5%
        idx = fps(pos, batch, ratio=0.375)
        x, pos, batch = x[idx], pos[idx], batch[idx]
        # Second XConv
        x = F.relu(self.conv2(x, pos, batch))

        # Farthest point samplling, keepiong only 33.4%
        idx = fps(pos, batch, ratio=0.334)
        x, pos, batch = x[idx], pos[idx], batch[idx]

        # Two additional XConvs
        x = F.relu(self.conv3(x, pos, batch))
        x = F.relu(self.conv4(x, pos, batch))

        # Pooling batch-elements together
        # Each tree is described in one single point with 384 features
        x = global_mean_pool(x, batch)

        # MLPs at the end with ReLU
        x = F.relu(self.lin1(x))
        x = F.relu(self.lin2(x))

        # Dropout: Set randomly to value of zero
        # x = F.dropout(x, p=0.5, training=self.training)
        x = F.dropout(x, p=0.5, training=True)
        return self.lin3(x)

# Main

In [ ]:
import logging
import os
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torchvision

# from models.pointcnn import PointCNN
from sklearn.metrics import classification_report, confusion_matrix
from tensorboardX import SummaryWriter
from torch_geometric.loader import DataLoader
from tqdm import tqdm

# from utils.tools import (
#     IOStream,
#     PointCloudsInFiles,
#     _init_,
#     delete_files,
#     make_confusion_matrix,
# )

In [ ]:
# Path to datasets
train_dataset_path = r"D:\MurrayBrent\data\RMF_ITD\PLOT_LAS\BUF_5M_SC\train"
val_dataset_path = r"D:\MurrayBrent\data\RMF_ITD\PLOT_LAS\BUF_5M_SC\val"
test_dataset_path = r"D:\MurrayBrent\data\RMF_ITD\PLOT_LAS\BUF_5M_SC\test"


# Load pretrained model ("" if training)
# pretrained = r"D:\MurrayBrent\git\point-dl\notebooks\checkpoints\PointCNN_2048_6\models\best_model.t7"
pretrained = ""

# max_points
# 1024, 2048, 3072, 4096, 5120, 6144, 7168, 8192, 9216, 10240,
# 11264, 12288, 13312, 14336, 15360, 16384, 17408, 18432, 19456, 20480
max_points = 2048

# Fields to include from pointcloud
use_columns = ["intensity"]

# Classes: must be in same order as in data
# classes = ["Con", "Dec"]
classes = [
    "Jack Pine",
    "White Spruce",
    "Black Spruce",
    # "Balsam Fir",
    # "Eastern White Cedar",
    "American Larch",
    "Paper Birch",
    "Trembling Aspen",
]

# Model Name
model_name = f"PointCNN_{max_points}_{len(classes)}"

In [ ]:
def test_one_epoch(device, model, test_loader, testing=False):
    model.eval()  # https://stackoverflow.com/questions/60018578/what-does-model-eval-do-in-pytorch
    test_loss = 0.0
    pred = 0.0
    count = 0
    y_pred = torch.tensor([], device=device)  # empty tensor
    y_true = torch.tensor([], device=device)  # empty tensor
    outs = torch.tensor([], device=device)  # empty tensor

    # Iterate through data in loader
    for i, data in enumerate(
        tqdm(test_loader, desc="Validation", leave=False, colour="green")
    ):
        # Send data to defined device
        data.to(device)

        # Call model
        output = model(data)

        # Define validation loss using negative log likelihood loss and softmax
        loss_val = torch.nn.functional.nll_loss(
            torch.nn.functional.log_softmax(output, dim=1),
            target=data.y,
        )

        # Update test_lost and count
        test_loss += loss_val.item()
        count += output.size(0)

        # Update pred and true
        _, pred1 = output.max(dim=1)
        ag = pred1 == data.y
        am = ag.sum()
        pred += am.item()

        y_true = torch.cat((y_true, data.y), 0)  # concatentate true values
        y_pred = torch.cat((y_pred, pred1), 0)  # concatenate predicted values
        outs = torch.cat((outs, output), 0)  # concatentate output

    # Calculate test_loss and accuracy
    test_loss = float(test_loss) / count
    accuracy = float(pred) / count

    # For validation
    if testing is False:
        # Create confusion matrix and classification report
        y_true = y_true.cpu().numpy()  # convert to array and send to cpu
        y_pred = y_pred.cpu().numpy()  # convert to array and send to cpu
        conf_mat = confusion_matrix(y_true, y_pred)  # create confusion matrix
        cls_rpt = classification_report(  # create classification report
            y_true,
            y_pred,
            target_names=classes,
            labels=np.arange(len(classes)),
            output_dict=True,
            zero_division=1,
        )
        return test_loss, accuracy, conf_mat, cls_rpt

    # For testing
    else:
        # out = torch.nn.functional.log_softmax(output, dim=1)  # softmax of output
        out = torch.nn.functional.softmax(outs, dim=1)
        return test_loss, accuracy, out

In [ ]:
def test(device, model, test_loader, textio):
    # Run test_one_epoch with testing as true
    test_loss, test_accuracy, out = test_one_epoch(
        device, model, test_loader, testing=True
    )

    # Print and save loss and accuracy
    textio.cprint(
        "Testing Loss: %f & Testing Accuracy: %f" % (test_loss, test_accuracy)
    )

    return out

In [ ]:
def train_one_epoch(device, model, train_loader, optimizer, epoch_number):
    model.train()
    train_loss = 0.0
    pred = 0.0
    count = 0

    # Iterate through data in loader
    for i, data in enumerate(
        tqdm(
            train_loader, desc="Epoch: " + str(epoch_number), leave=False, colour="blue"
        )
    ):
        # Send data to device
        data.to(device)

        # Call model
        output = model(data)

        # Define validation loss using negative log likelihood loss and softmax
        loss_val = torch.nn.functional.nll_loss(
            torch.nn.functional.log_softmax(output, dim=1),
            target=data.y,
        )

        # Forward + backward + optimize
        optimizer.zero_grad()
        loss_val.backward()
        optimizer.step()

        # Update train_loss and count
        train_loss += loss_val.item()
        count += output.size(0)

        # Update pred
        _, pred1 = output.max(dim=1)
        ag = pred1 == data.y
        am = ag.sum()
        pred += am.item()

    # Calculate train_loss and accuracy
    train_loss = float(train_loss) / count
    accuracy = float(pred) / count

    return train_loss, accuracy

In [ ]:
def train(
    device,
    model,
    train_loader,
    test_loader,
    boardio,
    textio,
    checkpoint,
    model_name,
    optimizer="Adam",
    start_epoch=0,
    epochs=200,
):
    # Set up optimizer
    learnable_params = filter(lambda p: p.requires_grad, model.parameters())
    if optimizer == "Adam":  # Adam optimizer
        optimizer = torch.optim.Adam(learnable_params)
    else:  # SGD optimizer
        optimizer = torch.optim.SGD(learnable_params, lr=0.1)

    # Set up checkpoint
    if checkpoint is not None:
        min_loss = checkpoint["min_loss"]
        optimizer.load_state_dict(checkpoint["optimizer"])

    # Define best_test_loss
    best_test_loss = np.inf

    # Run for every epoch
    for epoch in tqdm(
        range(start_epoch, epochs), desc="Total", leave=False, colour="red"
    ):
        # Train Model
        train_loss, train_accuracy = train_one_epoch(
            device, model, train_loader, optimizer, epoch + 1
        )

        # Validate model: testing=False
        test_loss, test_accuracy, conf_mat, cls_rpt = test_one_epoch(
            device, model, test_loader, testing=False
        )

        # Save Best Model
        if test_loss < best_test_loss:
            best_test_loss = test_loss

            # Create snap dictionary
            snap = {
                # state_dict: https://pytorch.org/tutorials/recipes/recipes/what_is_state_dict.html
                "epoch": epoch + 1,
                "model": model.state_dict(),
                "min_loss": best_test_loss,
                "optimizer": optimizer.state_dict,
            }
            # Save best snap dictionary
            torch.save(snap, f"checkpoints/{model_name}/models/best_model_snap.t7")

            # Save best model
            torch.save(
                model.state_dict(), f"checkpoints/{model_name}/models/best_model.t7"
            )

            # Make confusion matrix figure
            make_confusion_matrix(conf_mat, categories=classes)

            # Save best model confusion matrix
            delete_files(
                f"checkpoints/{model_name}/confusion_matrix/best", "*.png"
            )  # delete previous

            plt.savefig(
                f"checkpoints/{model_name}/confusion_matrix/best/confusion_matrix_epoch{epoch+1}.png",
                bbox_inches="tight",
                dpi=300,
            )  # save png

            plt.close()

            # Create classification report figure
            sns.heatmap(pd.DataFrame(cls_rpt).iloc[:-1, :].T, annot=True, cmap="Blues")

            # save best model classification report
            delete_files(
                f"checkpoints/{model_name}/classification_report/best", "*.png"
            )  # delete previous

            plt.savefig(
                f"checkpoints/{model_name}/classification_report/best/classification_report_epoch{epoch+1}.png",
                bbox_inches="tight",
                dpi=300,
            )  # save png

            plt.close()

        # Create confusion matrix figure
        make_confusion_matrix(conf_mat, categories=classes)
        plt.savefig(
            f"checkpoints/{model_name}/confusion_matrix/all/confusion_matrix_epoch{epoch+1}.png",
            bbox_inches="tight",
            dpi=300,
        )  # save .png

        plt.close()

        # Create classification report figure
        sns.heatmap(pd.DataFrame(cls_rpt).iloc[:-1, :].T, annot=True, cmap="Blues")
        plt.savefig(
            f"checkpoints/{model_name}/classification_report/all/classification_report_epoch{epoch+1}.png",
            bbox_inches="tight",
            dpi=300,
        )  # save .png
        plt.close()

        # Save most recent model
        torch.save(snap, f"checkpoints/{model_name}/models/model_snap.t7")
        torch.save(model.state_dict(), f"checkpoints/{model_name}/models/model.t7")

        # Add values to TensorBoard
        boardio.add_scalar("Train Loss", train_loss, epoch + 1)  # training loss
        boardio.add_scalar("Validation Loss", test_loss, epoch + 1)  # validation loss
        boardio.add_scalar(
            "Best Validation Loss", best_test_loss, epoch + 1
        )  # best validation loss
        boardio.add_scalar(
            "Train Accuracy", train_accuracy, epoch + 1
        )  # training accuracy
        boardio.add_scalar(
            "Validation Accuracy", test_accuracy, epoch + 1
        )  # validation accuracy
        boardio.add_scalars(
            "Loss",
            {"Training Loss": train_loss, "Validation Loss": test_loss},
            epoch + 1,
        )  # training and validation loss

        # Print and save losses and accuracies
        textio.cprint(
            "EPOCH:: %d, Training Loss: %f, Validation Loss: %f, Best Loss: %f"
            % (epoch + 1, train_loss, test_loss, best_test_loss)
        )
        textio.cprint(
            "EPOCH:: %d, Training Accuracy: %f Validation Accuracy: %f"
            % (epoch + 1, train_accuracy, test_accuracy)
        )

In [ ]:
def main(pretrained="", augment=True):
    # Set up TensorBoard summary writer
    boardio = SummaryWriter(log_dir="checkpoints/" + model_name)
    _init_(model_name)

    # Set up logger
    textio = IOStream("checkpoints/" + model_name + "/run.log")
    textio.cprint(model_name)

    # Get training, validation and test datasets
    if train_dataset_path:
        trainset = PointCloudsInFiles(
            train_dataset_path,
            "*.laz",
            "Class",
            max_points=max_points,
            use_columns=use_columns,
        )

        # Augment training data
        if augment is True:
            aug_trainset = AugmentPointCloudsInFiles(
                train_dataset_path,
                "*.laz",
                "Class",
                max_points=max_points,
                use_columns=use_columns,
            )

            # Concat training and augmented training datasets
            trainset = torch.utils.data.ConcatDataset([trainset, aug_trainset])
        # Load training dataset
        train_loader = DataLoader(trainset, batch_size=32, shuffle=True, num_workers=0)

    if val_dataset_path:
        valset = PointCloudsInFiles(
            val_dataset_path,
            "*.laz",
            "Class",
            max_points=max_points,
            use_columns=use_columns,
        )
        # Load validation dataset
        val_loader = DataLoader(valset, batch_size=32, shuffle=False, num_workers=0)

    if test_dataset_path:
        testset = PointCloudsInFiles(
            test_dataset_path,
            "*.laz",
            "Class",
            max_points=max_points,
            use_columns=use_columns,
        )
        # Load testing dataset
        test_loader = DataLoader(testset, batch_size=32, shuffle=False, num_workers=0)

    # Define device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Define model
    model = PointCNN(numfeatures=len(use_columns), numclasses=len(classes))

    # Checkpoint
    checkpoint = None

    # Load existing model
    if pretrained:
        assert os.path.isfile(pretrained)
        model.load_state_dict(torch.load(pretrained, map_location="cpu"))

    # Send model to defined device
    model.to(device)

    # Run testing
    if pretrained:
        finished = test(
            device=device, model=model, test_loader=test_loader, textio=textio
        )
        return finished
    # Run training
    else:
        train(
            device=device,
            model=model,
            model_name=model_name,
            train_loader=train_loader,
            test_loader=val_loader,
            boardio=boardio,
            textio=textio,
            checkpoint=checkpoint,
        )

In [ ]:
# Runtime
if __name__ == "__main__":
    main(pretrained=pretrained)
    # value = main(pretrained=pretrained)

# TESTING

In [ ]:
value = value.cpu().detach().numpy()

In [ ]:
range(len(value))

In [ ]:
files = list(Path(r"D:\MurrayBrent\data\RMF_ITD\PLOT_LAS\BUF_5M_SC\test").glob("*.laz"))
for i in range(len(value)):
    print(files[i])
    print(value[i])
    print(" ")